In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

tf.test.is_gpu_available()

/Users/zhenwuzhou/environment/tf1_py3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/zhenwuzhou/environment/tf1_py3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/zhenwuzhou/environment/tf1_py3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/zhenwuzhou/environment/tf1_py3

1.13.1
sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)
matplotlib 3.1.2
numpy 1.18.1
pandas 0.25.3
sklearn 0.22.1
tensorflow 1.13.1
tensorflow._api.v1.keras 2.2.4-tf


False

# Tf1.0
### Tf1.0实现全连接网络
```python
placeholder,session,feed_dict
1、placeholder, tf.layers.dense, tf.train.AdamOptimizer
2、tf.losses.sparse_softmax_cross_entropy
3、Tf.global_variables_initializer,feed_dict
```
### Dataset使用
```python
1、Dataset.make_one_shot_iterator
2、Dataset.make_initializable_iterator
```
### 自定义estimator
```python
1、Tf.feature_column.input_layer
2、Tf.estimator.EstimatorSpec
3、Tf.metrics.accuracy
```

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all,y_train_all),(x_test, y_test) = fashion_mnist.load_data()
x_valid,x_train = x_train_all[:5000],x_train_all[5000:]
y_valid,y_train = y_train_all[:5000],y_train_all[5000:]

print(x_train.shape,y_train.shape)
print(x_valid.shape,y_valid.shape)
print(x_test.shape,y_test.shape)
print(np.max(x_train),np.min(x_train))

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# x_train:[None,28,28] -> [None,784]
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)
x_valid_scaled = scaler.fit_transform(
    x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)
x_test_scaled = scaler.fit_transform(
    x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)
print(np.max(x_train_scaled),np.min(x_train_scaled))

(55000, 28, 28) (55000,)
(5000, 28, 28) (5000,)
(10000, 28, 28) (10000,)
255 0
2.0231433 -0.8105136


In [3]:
# 构建模型
hidden_units = [100,100]
class_num = 10 # 10分类

x = tf.placeholder(tf.float32,[None,28 * 28])
y = tf.placeholder(tf.int64, [None])

input_for_next_layer = x
# 隐藏层
for hidden_unit in hidden_units:
    input_for_next_layer = tf.layers.dense(input_for_next_layer,
                                           hidden_unit,
                                           activation=tf.nn.relu)
# 输出层
logits = tf.layers.dense(input_for_next_layer,
                         class_num)

# loss计算
# 1. 将最后一层的输出做softmax计算出prob(每种分类的概率)
# 2. labels -> one_hot 将标签进行one-hot编码
# 3. 计算loss：就是去概率最大的那个max_prob进行-log(max_prob)的计算
loss = tf.losses.sparse_softmax_cross_entropy(labels = y,
                                              logits = logits)

# 计算正确率accuracy
# 取出概率最大的那个位置的索引位置作为预测
prediction = tf.argmax(logits, 1)
correct_prediction = tf.equal(prediction,y) # 正确的为1，错误的为0
# 先将int值转为float类型然后在进行正确率的计算
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64)) 

# 定义train_op,在训练的时候就是调用train_op
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [4]:
print(x)
print(logits)

Tensor("Placeholder:0", shape=(?, 784), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(?, 10), dtype=float32)


In [6]:
# 模型训练
init = tf.global_variables_initializer()
batch_size = 32
epochs = 10
train_steps_per_epoch = x_train_scaled.shape[0] // batch_size
valid_steps = x_valid_scaled.shape[0] // batch_size

# 定义计算正确率的方法
def eval_with_sess(sess,x,y,accuracy,images,labels,batch_size):
    eval_steps = images.shape[0] // batch_size
    eval_accuracies = []
    for step in range(eval_steps):
        batch_data = images[step * batch_size : (step+1) * batch_size]
        batch_label = labels[step * batch_size : (step+1) * batch_size]
        accuracy_val = sess.run(accuracy,
                                feed_dict = {
                                    x:batch_data,
                                    y:batch_label
                                })
        eval_accuracies.append(accuracy_val)
    return np.mean(eval_accuracies)

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        for step in range(train_steps_per_epoch):
            batch_data = x_train_scaled[step * batch_size : (step+1) * batch_size]
            batch_label = y_train[step * batch_size : (step+1) * batch_size]
            # 同时run多个op
            loss_val,accuracy_val,_ = sess.run(
                [loss,accuracy,train_op],
                feed_dict = { # 在此次训练时要填入的数据
                    x:batch_data,
                    y:batch_label
                })
            print('\r[Train] epoch: %d, step: %d, loss: %3.5f, accuracy: %2.2f' % 
                  (epoch,step,loss_val,accuracy_val),end="")
        valid_accuracy = eval_with_sess(sess,x,y,accuracy,
                                        x_valid_scaled,y_valid,
                                        batch_size= batch_size)
        print("\t[Valid] acc: %2.2f" % valid_accuracy)

[Train] epoch: 0, step: 269, loss: 0.82103, accuracy: 0.75

[Train] epoch: 0, step: 538, loss: 0.46001, accuracy: 0.88

[Train] epoch: 0, step: 816, loss: 0.56444, accuracy: 0.72

[Train] epoch: 0, step: 1098, loss: 0.51186, accuracy: 0.78

[Train] epoch: 0, step: 1376, loss: 0.44801, accuracy: 0.81

[Train] epoch: 0, step: 1651, loss: 0.17363, accuracy: 0.94

[Train] epoch: 0, step: 1717, loss: 0.44631, accuracy: 0.88	[Valid] acc: 0.86
[Train] epoch: 1, step: 1717, loss: 0.45650, accuracy: 0.88	[Valid] acc: 0.87
[Train] epoch: 2, step: 1717, loss: 0.43978, accuracy: 0.88	[Valid] acc: 0.87
[Train] epoch: 3, step: 1717, loss: 0.37902, accuracy: 0.88	[Valid] acc: 0.87
[Train] epoch: 4, step: 1717, loss: 0.32899, accuracy: 0.84	[Valid] acc: 0.87
[Train] epoch: 5, step: 1717, loss: 0.30481, accuracy: 0.91	[Valid] acc: 0.87
[Train] epoch: 6, step: 1717, loss: 0.26736, accuracy: 0.88	[Valid] acc: 0.88
[Train] epoch: 7, step: 1717, loss: 0.20959, accuracy: 0.91	[Valid] acc: 0.88
[Train] epoch: 8, step: 1717, loss: 0.17004, accuracy: 0.91	[Valid] acc: 0.88
[Train] epoch: 9, step: 1717, loss: 0.23340, accuracy: 0.88	[Valid] acc: 0.88
